<a href="https://colab.research.google.com/github/RutulPatel007/VR_Project1_Rutul_Patel_IMT2022021/blob/main/MP_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install opencv-python numpy scikit-learn tensorflow keras matplotlib

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models

In [3]:
# Paths
data_dir = "drive/MyDrive/MP_VR"  # Change to your dataset path
categories = ["with_mask", "without_mask"]


In [6]:
# Function to extract HOG features
def extract_hog_features(image):
    image = cv2.resize(image, (64, 128))  # Resize for HOG compatibility
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor()
    return hog.compute(gray).flatten()


In [7]:
# Load dataset and extract handcrafted features
data, labels = [], []
for category in categories:
    path = os.path.join(data_dir, category)
    label = 0 if category == "without_mask" else 1
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        features = extract_hog_features(img)
        data.append(features)
        labels.append(label)


In [9]:
# Convert to numpy arrays
data, labels = np.array(data), np.array(labels)


In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
# Train SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))


SVM Accuracy: 0.8867235079171741


In [12]:
# Train Neural Network classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
mlp.fit(X_train, y_train)
mlp_preds = mlp.predict(X_test)
print("MLP Accuracy:", accuracy_score(y_test, mlp_preds))


MLP Accuracy: 0.9269183922046285


# CNN

In [13]:
# CNN Model
def create_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [14]:
# Load dataset for CNN
cnn_data, cnn_labels = [], []
for category in categories:
    path = os.path.join(data_dir, category)
    label = 0 if category == "without_mask" else 1
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        cnn_data.append(img)
        cnn_labels.append(label)


In [15]:
cnn_data, cnn_labels = np.array(cnn_data) / 255.0, np.array(cnn_labels)
X_train, X_test, y_train, y_test = train_test_split(cnn_data, cnn_labels, test_size=0.2, random_state=42)


In [16]:
cnn = create_cnn()
cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.7604 - loss: 0.5044 - val_accuracy: 0.9001 - val_loss: 0.2654
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9071 - loss: 0.2624 - val_accuracy: 0.9220 - val_loss: 0.2123
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9282 - loss: 0.1910 - val_accuracy: 0.9172 - val_loss: 0.2229
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9461 - loss: 0.1453 - val_accuracy: 0.8940 - val_loss: 0.2817
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9518 - loss: 0.1272 - val_accuracy: 0.9318 - val_loss: 0.2236
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9641 - loss: 0.1133 - val_accuracy: 0.9269 - val_loss: 0.2064
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9745 - loss: 0.0661 - val_accuracy: 0.9330 - val_loss: 0.2038
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9828 - loss: 0.0554 - val_accuracy: 0

In [17]:
# Evaluate CNN
cnn_acc = cnn.evaluate(X_test, y_test)[1]
print("CNN Accuracy:", cnn_acc)


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9537 - loss: 0.1897
CNN Accuracy: 0.9524969458580017
